In [89]:
## Visualizing the stock market structure
## Source: http://scikit-learn.org/stable/auto_examples/applications/plot_stock_market.html#stock-market

import os
import pandas as pd
import numpy as np
from sklearn import cluster, covariance, manifold

%matplotlib inline

In [90]:
def pandasReader(io):
    """Return a series of the weekly variation of historical data."""
    # Open the file
    df = pd.read_csv(io)
    
    # Set time column as index and make sure it is a date format
    df = df.set_index('Date')
    df.index = pd.to_datetime(df.index)
    
    # Extract the Close column only
    df = df[['Open', 'Close']]
    
    # Opening price on Monday
    df1 = df.groupby(pd.Grouper(freq='W-MON'))[['Open']].first()
    
    # We sync the df1 index on Friday to easily merge it with the closing date dataframe
    df1 = df1.groupby(pd.Grouper(freq='W-FRI'))[['Open']].first()
    
    # Closing price on Friday
    df2 = df.groupby(pd.Grouper(freq='W-FRI'))[['Close']].last()
    
    # Concatenate those 2 frames
    df3 = pd.concat([df1, df2], axis=1)
    
    # Create a variation column
    df3['Variation'] = df3['Close'] - df3['Open']
    
    # Create a return column
    df3['%Return'] = df3['Variation'].pct_change()
    
    # Create a rolling median column
    df3['Median'] = df3['%Return'].rolling(2, min_periods=1).median()
    
    # Create a return column that replaces inf values by the rolling median
    df3['Return'] = np.where(df3['%Return'].isin([-np.inf, np.inf]), df3['Median'], df3['%Return'])
        
    return df3[['Return']] 

In [91]:
def quotesReader(path, progress=False):
    """Get the historical weekly returns from data in various csv files. """
    symbols = []
    frames = []
    
    # List of csv files we are going to exclude from our analysis
    rejected_symbols = ['NLSN', 'QRVO', 'CHTR', 'HCA', 'CBOE', 'CFG', 'PSX', 'IQV', 'NAVI',
                        'NWS', 'FTV', 'MPC', 'KHC', 'ALLE', 'ABBV', 'KMI', 'APTV', 'KORS', 'GM',
                        'VRSK', 'PYPL', 'AVY', 'WRK', 'ZTS', 'DG', 'HPE', 'TRIP', 'XYL', 'FBHS',
                        'CBRE', 'HII', 'HST', 'EVHC', 'COTY', 'AIV', 'AVGO', 'UA', 'LYB', 'SYF',
                        'INFO', 'NCLH', 'AIG', 'NWSA', 'HLT']
    
    # Open each csv file in path
    for i, filename in enumerate(os.listdir(path)):
        
        # Print progress rate
        if progress:
            progress_rate = int(i / len(os.listdir(path)) * 100)
            print('# Progress :', str(progress_rate) + '%')
            
        
        # Extract the symbol from the filename
        symbol = filename.split('.csv')[0]
        
        # Only take into account the non-rejected symbols
        if not symbol in rejected_symbols:
            
            # Add it to the symbol list
            symbols.append(symbol)
            
            # Get the path of the csv file
            filepath = os.path.join(path, filename)

            # Extract the weekly returns using our pandasReader function and add it to the list of frames
            df = pandasReader(filepath)
            frames.append(df)
        
        
    # Concat all the frames horizontally and name the columns with the corresponding symbols
    df1 = pd.concat(frames, axis=1)
    df1.columns = symbols
    
    # Remove rows containing NaN values
    df1 = df1.dropna(axis=0)
    
    # Save as excel file
    writer = pd.ExcelWriter('output.xlsx')
    df1.to_excel(writer,'Sheet1')
    writer.save()
    
    
    
    return df1

df = quotesReader('Stocks')

In [92]:
df.head(10)

,CSCO,UAL,TROW,ISRG,PRGO,LUK,TPR,DVN,MRO,BA,...,TSS,CRM,PGR,WAT,BWA,LRCX,NWL,UAA,BLK,PPL
Date,,,,,,,,,,,,,,,,,,,,,
2012-05-25,-3.571465,-1.809836,-1.132479,-1.362843,-3.027171,-1.014779,-2.927151,-0.279817,-1.650000,-0.877006,...,-3.164553,-0.693617,-1.478264,-1.036424,-1.149734,-1.174698,-3.755568,-2.845382,-1.830079,-3.076923
2012-06-01,2.055556,-0.757085,-9.612854,-2.679431,-1.541554,-42.332329,18.436400,0.707004,-2.605768,5.608712,...,-1.672515,-6.545133,-3.136344,-51.818218,-16.553419,-5.275877,-1.854838,-1.408052,-2.543529,-2.037037
2012-06-08,-2.363636,-2.066663,-2.348313,-2.301778,-3.242577,-1.572581,-0.986209,-1.257463,-1.634732,-1.588816,...,-1.808697,-1.398247,-2.297870,-1.291591,-1.283581,-1.600806,-1.292453,-3.016000,-2.027440,-2.642857
2012-06-15,-0.693335,1.125003,-0.624999,-1.263579,0.105961,-0.422534,-5.358983,-3.942038,-1.424527,-0.229052,...,-0.731183,-1.435535,-2.770495,-0.288345,-2.376519,-1.053693,0.032258,-0.845238,-0.799705,-1.239133
2012-06-22,-0.782612,-2.375001,-0.711112,-3.163295,-0.325350,-1.121950,-0.632353,-0.221675,1.022227,-0.804350,...,-1.120004,-3.090247,-0.694444,0.025860,-1.308825,-3.999950,-2.312500,-5.555556,-3.859279,3.818145
2012-06-29,2.200064,-0.818182,6.564125,-0.473713,0.044379,-10.000103,-0.544001,-3.018987,-3.384615,8.222343,...,-19.332756,-0.473231,-1.939394,-1.663870,0.323804,0.625003,-1.714283,-0.275797,-1.183939,-1.792451
2012-07-06,-3.125006,-1.558824,-1.454237,-1.494562,-0.943341,-1.022222,1.105262,-1.043887,-1.276498,-1.208834,...,-0.654544,-1.380331,-1.290323,1.734178,-0.431655,-7.384545,-0.799996,-0.595855,1.070406,-1.166664
2012-07-13,0.235293,-0.421047,-1.679104,1.461993,-16.799625,-114.001130,0.258336,11.214366,-0.983337,0.461537,...,-3.894716,3.560322,4.666667,-0.833333,-1.354441,-0.638554,-9.833169,-2.942308,1.904763,-9.571560
2012-07-20,-1.357147,21.272534,-1.945056,2.415921,-0.632911,-1.690267,-2.271525,-3.163745,-151.030006,-2.000009,...,-2.127271,-1.984880,-1.274512,-1.527782,0.035670,-1.877778,-1.886796,0.610561,-1.515221,-0.099998


In [93]:
# Learn a graphical structure from the correlations
edge_model = covariance.GraphLassoCV()

# standardize the time series: using correlations rather than covariance
# is more efficient for structure recovery
X = df.copy()
X /= X.std(axis=0)
edge_model.fit(X)

# Find a low-dimension embedding for visualization: find the best position of
# the nodes (the stocks) on a 2D plane

# We use a dense eigen_solver to achieve reproducibility (arpack is
# initiated with random vectors that we don't control). In addition, we
# use a large number of neighbors to capture the large-scale structure.
node_position_model = manifold.LocallyLinearEmbedding(
    n_components=2, eigen_solver='dense', n_neighbors=6)

embedding = node_position_model.fit_transform(X.T).T

# Cluster using affinity propagation

_, labels = cluster.affinity_propagation(edge_model.covariance_)
n_labels = labels.max()
names = np.array(df.columns)

for i in range(n_labels + 1):
    print('Cluster %i: %s' % ((i + 1), ', '.join(names[labels == i])))

Cluster 1: TROW, AYI
Cluster 2: ISRG, A, SRCL, OXY, SPG, IBM
Cluster 3: LUK, HAS
Cluster 4: MRO, CVS, HBAN
Cluster 5: BA, TMO, CHRW, CAG
Cluster 6: DVN, V, DRE, XLNX, KSS, PHM, NUE
Cluster 7: FOX, GLW, CME
Cluster 8: MSCI, INTU, ILMN
Cluster 9: EIX, XRX
Cluster 10: C, BIIB, GPC
Cluster 11: T, XRAY, NTRS, AMGN
Cluster 12: CF, MMM, HIG, GWW, CHD
Cluster 13: MO, MGM, D, XEC
Cluster 14: XL
Cluster 15: CSCO, HUM, STZ, CERN
Cluster 16: WU, KR, ADP
Cluster 17: MSI, COST, WELL, IP, CVX, ZION
Cluster 18: FCX, ULTA, APD
Cluster 19: EQIX, DTE, UTX, ROP, EOG, GE, MHK, UAA
Cluster 20: TPR, PKI, ADI, AON, FRT, MCK
Cluster 21: AJG, LNC, PWR
Cluster 22: PPG, BKNG, HPQ, NVDA, KIM, PGR
Cluster 23: FLS, ESRX, DVA
Cluster 24: ARE, AZO
Cluster 25: SYK, IPG, SO
Cluster 26: TSN, MCO
Cluster 27: PEP, DHR, GRMN
Cluster 28: REG, AMG, LB, RHI
Cluster 29: SWKS, LLL, VRSN, BLL
Cluster 30: MDLZ, FAST, MAR
Cluster 31: ADM, WYN, FMC, K
Cluster 32: AIZ, UNP, TMK
Cluster 33: DLR, CAT, EXPE, ACN, RE, MA
Cluster 34: LNT,